In [3]:
import pandas as pd
from transformers import BertTokenizerFast, BertForSequenceClassification, get_linear_schedule_with_warmup, AdamW
from sklearn.model_selection import train_test_split
import torch
from tqdm import tqdm 
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler, TensorDataset
import os
import numpy as np
import random
from sklearn.metrics import accuracy_score
import json
from torch.utils.data import DataLoader
from sentence_transformers import SentenceTransformer, models, losses, InputExample

In [4]:
df = pd.read_csv("../train.csv") #Сюда нужно вставить ссылку на данные с гугл - диска.

In [5]:
df.head()

,pair_id,name_1,name_2,is_duplicate
0,1,Iko Industries Ltd.,"Enormous Industrial Trade Pvt., Ltd.",0
1,2,Apcotex Industries Ltd.,Technocraft Industries (India) Ltd.,0
2,3,"Rishichem Distributors Pvt., Ltd.",Dsa,0
3,4,Powermax Rubber Factory,Co. One,0
4,5,Tress A/S,Longyou Industries Park Zhejiang,0


In [6]:
filtered_df_0 = df[df['is_duplicate']==0].sample(4000)
filtered_df_1 = df[df['is_duplicate']==1]
filtered_df = pd.concat([filtered_df_0, filtered_df_1], axis=0)

In [7]:

print(len(df))
print(len(filtered_df))
print(len(filtered_df_0))
print(len(filtered_df_1))
df.head()
filtered_df.head()
filtered_df_1.head()


497819
7658
4000
3658


,pair_id,name_1,name_2,is_duplicate
161,162,JX Nippon Oil & Gas Exploration (Brasil) Ltda,JX Nippon Oil & Gas Exploration Technical Serv...,1
603,604,Pirelli Neumaticos S.A.I.C.,"Pirelli Tyre Co., Ltd.",1
835,836,Brenntag Australia (Pty) Ltd.,Brenntag Group,1
1328,1329,"PAUL BAUDER GMBH & CO KG, BOCHUM PLANT",Paul Bauder ag,1
1562,1563,TOTAL CESKA REPUBLIKA s.r.o.,TOTAL FRANCE (ARNAY LE DUC),1


In [8]:
filtered_df_test = df[df['name_1']=="JX Nippon Oil & Gas Exploration (Brasil) Ltda"]
print(len(filtered_df_test))
filtered_df_test.tail()

10


,pair_id,name_1,name_2,is_duplicate
316074,316075,JX Nippon Oil & Gas Exploration (Brasil) Ltda,JX Nippon Oil & Gas Exploration (Myanmar) Limited,1
328496,328497,JX Nippon Oil & Gas Exploration (Brasil) Ltda,Nippon Oil Exploration U.S.A. Limited,1
389435,389436,JX Nippon Oil & Gas Exploration (Brasil) Ltda,Nippon Oil Exploration Limited,1
395094,395095,JX Nippon Oil & Gas Exploration (Brasil) Ltda,JX Nippon Oil & Gas Exploration (Malaysia) Lim...,1
444191,444192,JX Nippon Oil & Gas Exploration (Brasil) Ltda,JX Nippon Oil & Gas Exploration Corporation,1


In [10]:
def seed_everything(seed = 1234):
     random.seed(seed)
     os.environ['PYTHONHASHSEED'] = str(seed)
     np.random.seed(seed)     
     torch.manual_seed(seed)
     torch.cuda.manual_seed(seed)
     torch.backends.cudnn.deterministic = True

In [11]:
seed_everything()

In [12]:
triplet_dataset = json.load(open("../triplet_dataset.json")) #Сюда нужно вставить ссылку на данные с гугл - диска.

In [23]:
DEVICE = 'cuda:0'

In [ ]:
train_examples = []
test_examples = []
every_n_test = 10

for i, sample in tqdm(enumerate(triplet_dataset)):
  if i % every_n_test == 0:
    test_examples.append(InputExample(texts=[sample['set'], sample['pos'], sample['neg'][0]]))
    test_examples.append(InputExample(texts=[sample['pos'], sample['set'], sample['neg'][1]]))
    continue
  else:
    for j in range(len(sample['neg'])):
      if j % 2 == 0:
        train_examples.append(InputExample(texts=[sample['set'], sample['pos'], sample['neg'][j]]))
      else:
        train_examples.append(InputExample(texts=[sample['pos'], sample['set'], sample['neg'][j]]))
    


In [ ]:
print(f"We have a {type(train_examples)} of length {len(train_examples)} containing {type(train_examples[0])}'s.")

In [ ]:
print(f"We have a {type(test_examples)} of length {len(test_examples)} containing {type(test_examples[0])}'s.")

In [16]:
train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=)

In [ ]:


## Step 1: use an existing language model
word_embedding_model = models.Transformer('distilroberta-base')

#word_embedding_model = models.Transformer('stsb-roberta-large')
## Step 2: use a pool function over the token embeddings
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension())

## Join steps 1 and 2 using the modules argument
model = SentenceTransformer(modules=[word_embedding_model, pooling_model])

In [18]:


train_loss = losses.TripletLoss(model=model)

In [19]:
num_epochs = 5

warmup_steps = int(len(train_dataloader) * num_epochs * 0.1) #10% of train data

In [ ]:
DEVICE = 'cuda:0'
model.to(DEVICE)


In [ ]:
model.fit(train_objectives=[(train_dataloader, train_loss)],
          epochs=num_epochs,
          warmup_steps=warmup_steps) 

In [ ]:
torch.save(model.state_dict(), "model_state")
torch.save(model, "model")

In [ ]:
embeddings = model.encode(sentences, convert_to_tensor=True)

similarity = []
for i in range(len(sentences)):
    row = []
    for j in range(len(sentences)):
      row.append(util.pytorch_cos_sim(embeddings[i], embeddings[j]).item())
    similarity.append(row)